In [1]:
# necessary libraries for prediciton
import utils
import pandas as pd
import numpy as np
import pprint

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Useful Functions

In [2]:
def pretty_matrix(matrix, row_label, col_label):
    """Pretty print of the given matrix """

    # Restraining labels that are too big
    row_label = [el[:10] + '..' if len(el) > 10 else el
                for el in row_label]
    col_label = [el[:10] + '..' if len(el) > 10 else el
                for el in col_label]

    # Stringfying everything & Joining top label
    s_matrix = [list([" "] + (col_label))] + \
               [[row_label[row_idx]] + \
                [str(e) for e in row] for row_idx, row in enumerate(matrix)]

    # Length of each matrix column
    len_s = [max(map(len, col)) for col in zip(*s_matrix)]

    # Cell formatation
    formatation = '\t'.join('{{:{}}}'.format(x) for x in len_s)

    # Apply cell formation to each matrix element
    pretty_mat = [formatation.format(*row) for row in s_matrix]

    # Print Pretty Matrix
    print('\n'.join(pretty_mat))


def display_confusion_matrix(values):
    '''Display the given array as a confusion matrix'''
    pretty_matrix([values[0:2], values[2:4]],
                  ['Actual NO', 'Actual YES'],
                  ['Predic NO', 'Predic YES'])

In [3]:
def apply_PCA(df, variance_val=0.9, debug=True):
    '''Apply the PCA algorithm to given dataframe,
    using the given variance val to trim the df'''
    # Necessary to normalize all data to use PCA
    scaler=StandardScaler()
    X_scaled=scaler.fit_transform(df)

    # PCA - keep, by default mode, 90% variance
    pca = PCA(variance_val)    
    pca.fit(X_scaled)
    X_pca = pca.transform(X_scaled)

    if debug:
        ex_variance=np.var(X_pca,axis=0)
        ex_variance_ratio = ex_variance/np.sum(ex_variance)
        print(' > Impact in total variance of each generated feature by PCA:')
        print(ex_variance_ratio)

    principal_df = pd.DataFrame(data = X_pca)
    
    return (principal_df, pca)

In [4]:
def auc_scorer(y_true, y_pred):
    '''Scorer of Area Under Curve value'''
    fpr, tpr, _ = metrics.roc_curve(y_true, y_pred)
    return metrics.auc(fpr, tpr)

# Prediction Algorithms

* Decision Tree
* Random Forest
* Gradient Boosting

In [5]:
def create_DT():
    '''Create a new Decision Tree'''
    # Useful DecisionTree tutorial:
    # https://www.datacamp.com/community/tutorials/decision-tree-classification-python
    return DecisionTreeClassifier()

In [6]:
def create_RF():
    '''Create a new Ranfom Forest model'''
    return RandomForestClassifier()

In [7]:
def create_GB():
    '''Create a new Gradient Boosting model'''
    return GradientBoostingClassifier()

# Prediction

* Predictions are done in this notebook.
* It is also useful to compare how serveral algorithms perform against one another.

In [8]:
# Useful Macros
K_FOLD_NUM_SPLITS = 5
SEED = 42
USE_PCA = False

# Pretty printer
pp = pprint.PrettyPrinter(indent=4)

In [9]:
dataset =  utils.read_csv_to_df('dataset/preprocessed_data.csv')
display(dataset.head())

,date,amount,payments,frequency,account_creation_date,balance_mean,balance_max,balance_min,last_ballance,credit_mean,...,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_over_payments,status
0,0.000000,96396,8033,2,0.054011,12250.000000,20100.0,1100.0,20100.0,5025.000000,...,9650,3.38,3.67,100,2985,2804,105,0.685425,1617,-1
1,0.004721,165960,4610,1,0.024623,52083.859459,120512.8,700.0,52208.9,13523.158824,...,8369,1.79,2.31,117,2854,2618,148,0.241205,3759,1
2,0.018096,127080,2118,1,0.020651,30060.954167,49590.4,800.0,20272.8,5009.733333,...,8390,2.28,2.89,132,2080,2122,170,0.922457,6272,1
3,0.022817,105804,2939,1,0.013503,41297.480000,65898.5,1000.0,34307.3,9254.600000,...,10045,1.42,1.71,135,6604,6295,185,0.840626,7106,1
4,0.049567,274740,4579,2,0.025417,57188.211111,122893.1,600.0,41112.9,21255.930769,...,8288,3.79,4.52,110,1562,1460,204,0.030136,3709,1


In [10]:
STATUS_COL = dataset.columns.get_loc("status")

In [11]:
# Setting X and Y
X = dataset.iloc[:, 0:STATUS_COL]
y = dataset.iloc[:, [STATUS_COL]]
display(X.head())

if USE_PCA:
    print(' > Applying PCA to X_train: ')
    X, pca = apply_PCA(X, debug=True)
    display(X.head())

,date,amount,payments,frequency,account_creation_date,balance_mean,balance_max,balance_min,last_ballance,credit_mean,...,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_over_payments
0,0.000000,96396,8033,2,0.054011,12250.000000,20100.0,1100.0,20100.0,5025.000000,...,81.8,9650,3.38,3.67,100,2985,2804,105,0.685425,1617
1,0.004721,165960,4610,1,0.024623,52083.859459,120512.8,700.0,52208.9,13523.158824,...,73.5,8369,1.79,2.31,117,2854,2618,148,0.241205,3759
2,0.018096,127080,2118,1,0.020651,30060.954167,49590.4,800.0,20272.8,5009.733333,...,53.5,8390,2.28,2.89,132,2080,2122,170,0.922457,6272
3,0.022817,105804,2939,1,0.013503,41297.480000,65898.5,1000.0,34307.3,9254.600000,...,74.8,10045,1.42,1.71,135,6604,6295,185,0.840626,7106
4,0.049567,274740,4579,2,0.025417,57188.211111,122893.1,600.0,41112.9,21255.930769,...,50.5,8288,3.79,4.52,110,1562,1460,204,0.030136,3709


# Hyper Parameter Tunning

## Random Search 1st to apporach the best solution, GridSearch to refine it

In [12]:
def getDecisionTreeBest(X, y, debug=True):
    '''Get the Decision Tree Hyper Parameters'''

    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(2, 20, 4)]
    max_depth.append(None)

    # Create the random grid
    grid = {'criterion': ['gini', 'entropy'],
            'splitter': ['best', 'random'],
            'max_features': ['auto', 'sqrt'],
            'max_depth': max_depth,
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    dt = create_DT()
    dt_grid = GridSearchCV(estimator = dt,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    dt_grid = dt_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', dt_grid.best_score_)
        print('Best Params: ', dt_grid.best_params_)
        
    # Return score, method & params tuple
    return (dt_grid.best_score_, 'Decision Tree', dt_grid.best_params_)

In [13]:
def getRandomForestBest(X, y, debug=True):
    '''Get the Random Forest Hyper Parameters'''

    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(2, 20, 4)]
    max_depth.append(None)

    # Create the random grid
    grid = {'n_estimators': [int(x) for x in range(2, 14, 2)],
            'max_features': ['auto', 'sqrt'],
            'max_depth': max_depth,
            'criterion': ['gini', 'entropy'],
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4],
            'bootstrap': [True, False]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    rf = create_RF()
    rf_grid = GridSearchCV(estimator = rf,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    rf_grid = rf_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', rf_grid.best_score_)
        print('Best Params: ', rf_grid.best_params_)
        
    # Return score, method & params tuple
    return (rf_grid.best_score_, 'Random Forest', rf_grid.best_params_)

In [14]:
def getGradientBoostBest(X, y, debug=True):
    '''Get the Gradient Boost Hyper Parameters'''

    # Create the grid parameters
    grid = {'n_estimators': [int(x) for x in range(2, 14, 2)],
            'learning_rate': [0.1, 0.3, 0.5, 0.7],
            'loss': ['deviance', 'exponential'],
            'criterion': ['friedman_mse', 'mse', 'mae'],
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4],
            'random_state': [SEED]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    gb = create_GB()
    gb_grid = GridSearchCV(estimator = gb,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    gb_grid = gb_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', gb_grid.best_score_)
        print('Best Params: ', gb_grid.best_params_)
        
    # Return score, method & params tuple
    return (gb_grid.best_score_, 'Gradient Boosting', gb_grid.best_params_)

In [15]:
# Getting the best algorithm
algorithms = [getDecisionTreeBest(X, y),
              getRandomForestBest(X, y),
              getGradientBoostBest(X, y)]
algorithms.sort(reverse=True, key=lambda el: el[0])

for entry, index in enumerate(algorithms):
    print('%i. %s - %f\n---------' % (index, entry[1], entry[0]))
    
print('Best algorithm: %s\n\tConfig:\n' % algorithms[0][1])
pprint(algorithms[0][2])

{   'criterion': ['gini', 'entropy'],
    'max_depth': [2, 6, 10, 14, 18, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'splitter': ['best', 'random']}
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed:   11.4s finished
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:  0.7514640577561791
Best Params:  {'criterion': 'entropy', 'max_depth': 14, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'splitter': 'best'}
{   'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 6, 10, 14, 18, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [2, 4, 6, 8, 10, 12, 14, 16, 18]}
Fitting 5 folds for each of 3888 candidates, totalling 19440 fits


[Parallel(n_jobs=-1)]: Done 308 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 4196 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 7592 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 11972 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 16010 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 19172 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 19440 out of 19440 | elapsed:  4.1min finished
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/model_s

Best Score:  0.7771594573323553
Best Params:  {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 14, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 2}
{   'criterion': ['friedman_mse', 'mse', 'mae'],
    'learning_rate': [0.1, 0.3, 0.5, 0.7],
    'loss': ['deviance', 'exponential'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [2, 4, 6, 8, 10, 12, 14, 16, 18],
    'random_state': [42]}
Fitting 5 folds for each of 1944 candidates, totalling 9720 fits


[Parallel(n_jobs=-1)]: Done 458 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 2636 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 6290 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 7228 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 7958 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 8707 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 9234 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 9720 out of 9720 | elapsed:  5.2min finished
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/ensemble/gra

Best Score:  0.7293837681092976
Best Params:  {'criterion': 'friedman_mse', 'learning_rate': 0.5, 'loss': 'deviance', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 14, 'random_state': 42}


TypeError: 'NoneType' object is not iterable

## Using method with higher score with our data

In [18]:
# Cross validation settings
auc_scores = []
confusion_matrixes = []
cv = KFold(n_splits=K_FOLD_NUM_SPLITS, random_state=SEED, shuffle=False)

# CHANGE THIS LINE TO CHANGE THE USED CLASSIFICATION METHOD
# classifier = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2, min_samples_split=10, splitter='best')
# classifier = GradientBoostingClassifier(criterion='friedman_mse', learning_rate=0.7, loss='deviance',
#                                         min_samples_leaf=2, min_samples_split=2, n_estimators=8, random_state=SEED)
classifier = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=14, max_features='sqrt',
                                    min_samples_leaf=2, min_samples_split=5, n_estimators=2)


# Applying Cross validation
for train_index, test_index in cv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Training with this fold
    classifier.fit(X_train, y_train)
    
    # Testing & Measuring accuracy
    y_pred = classifier.predict(X_test)
    
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    auc_scores.append(metrics.auc(fpr, tpr))
    confusion_matrixes.append(metrics.confusion_matrix(y_test, y_pred).ravel())

/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionW

In [19]:
# Printing the obtained results
print('Classification Method used:', classifier, '\n')
print('AUC scores:', auc_scores)
print('> Average: ', sum(auc_scores)/len(auc_scores))
for cf in confusion_matrixes:
    display_confusion_matrix(cf)

Classification Method used: RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy',
                       max_depth=14, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=2,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) 

AUC scores: [0.5584795321637427, 0.7037037037037037, 0.7780172413793103, 0.6194968553459119, 0.7250000000000001]
> Average:  0.6769394665185338
          	Predic NO	Predic YES
Actual NO 	2        	7         
Actual YES	6        	51        
          	Predic NO	Predic YES
Actual NO 	6        	6         
Actual YES	5        	49        
          	Predic NO	Predic YES
Actual NO 	5        	3         
Actual YES	4        	54        
          	Predic NO	Predic YES
Actual NO 	4   

### After having our model trained we shall use the model on the data to be sumitted in the kaggle

In [20]:
test_dataset =  utils.read_csv_to_df('dataset/test_dataset.csv')
ids = utils.read_csv_to_df('dataset/ids.csv')
display(test_dataset.head())

,date,amount,payments,frequency,account_creation_date,balance_mean,balance_max,balance_min,last_ballance,credit_mean,...,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_over_payments,status
0,0.000000,93960,1566,1,0.176892,54520.202247,88246.7,800.0,49548.5,12769.220000,...,8390,2.28,2.89,132,2080,2122,452,0.389496,6824,NaN
1,0.001420,260640,7240,1,0.139059,31518.182051,88731.8,-718.6,11565.4,9526.772414,...,8620,1.10,1.25,100,1089,1117,490,0.688919,1380,NaN
2,0.007102,232560,4845,0,0.000000,40175.612500,79286.6,200.0,45754.0,15302.486047,...,10045,1.42,1.71,135,6604,6295,630,0.907412,5200,NaN
3,0.025568,221880,3698,1,0.339468,44440.912676,74216.8,1000.0,38913.4,11797.642857,...,8899,3.39,3.97,149,2987,2487,311,0.809056,5201,NaN
4,0.025568,38520,3210,1,0.047035,20231.313158,31302.0,900.0,18914.3,2736.026923,...,8388,2.41,2.94,87,1658,1668,597,0.894527,5178,NaN


In [21]:
# We now remove the Y column with NaNs
test_dataset = test_dataset.iloc[:, 0:STATUS_COL]
display(test_dataset.head())

,date,amount,payments,frequency,account_creation_date,balance_mean,balance_max,balance_min,last_ballance,credit_mean,...,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_over_payments
0,0.000000,93960,1566,1,0.176892,54520.202247,88246.7,800.0,49548.5,12769.220000,...,53.5,8390,2.28,2.89,132,2080,2122,452,0.389496,6824
1,0.001420,260640,7240,1,0.139059,31518.182051,88731.8,-718.6,11565.4,9526.772414,...,52.4,8620,1.10,1.25,100,1089,1117,490,0.688919,1380
2,0.007102,232560,4845,0,0.000000,40175.612500,79286.6,200.0,45754.0,15302.486047,...,74.8,10045,1.42,1.71,135,6604,6295,630,0.907412,5200
3,0.025568,221880,3698,1,0.339468,44440.912676,74216.8,1000.0,38913.4,11797.642857,...,55.3,8899,3.39,3.97,149,2987,2487,311,0.809056,5201
4,0.025568,38520,3210,1,0.047035,20231.313158,31302.0,900.0,18914.3,2736.026923,...,59.1,8388,2.41,2.94,87,1658,1668,597,0.894527,5178


In [28]:
if USE_PCA:
    # Using train PCA and classifying
    scaler=StandardScaler()
    X_test_scaled=scaler.fit_transform(test_dataset)
    predictions_df = pd.DataFrame(data = pca.transform(X_test_scaled))
    display(predictions_df)

    final_df = pd.DataFrame()
    final_df['Predicted'] = classifier.predict(predictions_df)
    final_df['Id'] = ids['loan_id']
    
else:
    predictions_df = test_dataset.copy()
    predictions_df['Predicted'] = classifier.predict(predictions_df)
    final_df = ids.merge(predictions_df, on=['date', 'amount'])
    final_df = final_df[['loan_id', 'Predicted']]\
                        .rename(columns={
                            'loan_id': 'Id'
                        })\
                        .drop_duplicates()

display(final_df)

,Id,Predicted
0,5895,1
1,7122,-1
2,6173,-1
3,6142,1
4,5358,-1
...,...,...
361,4989,-1
362,5221,1
363,6402,1
364,5346,1


In [29]:
# Outputting predictions to .csv
# CHANGE FILE NAME TO PRESERVE DIFFERENT INSTANCES
utils.write_df_to_csv(final_df, 'predictions', 'prediction.csv')